In [ ]:
# ----------------------------------------------------------------
# 1. 환경 설정 및 라이브러리 임포트
# ----------------------------------------------------------------
# 한글 폰트 설치 및 설정 (Matplotlib 시각화용)
!apt-get update -qq
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib
!pip install  -q easyocr


import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import easyocr
import numpy as np
import matplotlib.patches as patches
import cv2
from PIL import Image
import re
import json
import os

# 나눔 폰트 경로 확인 및 설정
font_dirs = ['/usr/share/fonts/truetype/nanum']
font_files = fm.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    fm.fontManager.addfont(font_file)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# Google Drive 마운트 (이미지 파일 접근용)
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------------------------------------------
# 2. 이미지 불러오기 및 경로 설정
# ----------------------------------------------------------------
# !!!! 중요: 아래 이미지 경로는 실제 파일 위치에 맞게 수정해주세요!!!!
image_path = '/content/drive/MyDrive/Classroom/shipdata/영수증 이미지 3.jpg' # 예시 경로

if not os.path.exists(image_path):
    print(f"에러: 이미지 파일을 찾을 수 없습니다. 경로를 확인해주세요: {image_path}")
    # 예외를 발생시키거나, 기본 이미지로 대체하는 등의 처리를 할 수 있습니다.
    # 여기서는 스크립트 실행을 중단합니다.
    raise FileNotFoundError(f"이미지 파일을 찾을 수 없습니다: {image_path}")

receipt_image_pil = Image.open(image_path)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done


In [ ]:


# ----------------------------------------------------------------
# 3. 이미지 전처리 함수 정의
# ----------------------------------------------------------------
def preprocess_receipt_image(pil_image):
    img_cv = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    img_cv = cv2.resize(img_cv, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((1, 1), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)
    enhanced = cv2.convertScaleAbs(eroded, alpha=1.2, beta=0)
    return Image.fromarray(enhanced) # PIL 이미지로 반환

# ----------------------------------------------------------------
# 4. 이미지 전처리 실행 및 결과 비교 시각화
# ----------------------------------------------------------------
processed_pil_image = preprocess_receipt_image(receipt_image_pil)

plt.figure(figsize=(12, 20))
plt.subplot(1, 2, 1)
plt.title("원본 이미지")
plt.imshow(receipt_image_pil)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("전처리된 이미지")
plt.imshow(processed_pil_image, cmap='gray')
plt.axis('off')
plt.tight_layout()
plt.show()

# ----------------------------------------------------------------
# 5. EasyOCR 실행 (전처리된 이미지 사용)
# ----------------------------------------------------------------
reader = easyocr.Reader(['ko', 'en'], gpu=True) # Colab에서는 GPU 사용 가능

ocr_result_on_processed = reader.readtext(
    np.array(processed_pil_image), # EasyOCR은 NumPy 배열을 입력으로 받음
    detail=1,
    paragraph=False,
    contrast_ths=0.1,
    adjust_contrast=0.8,
    text_threshold=0.6,
    width_ths=0.8,
    height_ths=0.8,
    decoder='beamsearch'
)

# ----------------------------------------------------------------
# 6. OCR 결과 추출 및 텍스트 후처리
# ----------------------------------------------------------------
extracted_texts = []
extracted_boxes_scaled_to_original = [] # 원본 이미지 크기 기준 박스 좌표

# 전처리 시 2배 확대했으므로, 원본 이미지에 박스를 그리려면 좌표를 0.5배 축소해야 함
scale_factor = 0.5

for detection in ocr_result_on_processed:
    box_on_processed, text, score = detection # box_on_processed는 전처리된(확대된) 이미지 기준 좌표
    if score > 0.2: # 낮은 신뢰도 결과도 일단 포함
        extracted_texts.append(text)

        # 전처리된 이미지 기준의 4개 꼭짓점 좌표
        # box_on_processed = [(x1,y1), (x2,y2), (x3,y3), (x4,y4)]
        x_coords = [point[0] for point in box_on_processed]
        y_coords = [point[1] for point in box_on_processed]

        # 원본 이미지 크기 기준으로 좌표 스케일링
        x_min_orig = min(x_coords) * scale_factor
        y_min_orig = min(y_coords) * scale_factor
        x_max_orig = max(x_coords) * scale_factor
        y_max_orig = max(y_coords) * scale_factor

        extracted_boxes_scaled_to_original.append(
            (int(x_min_orig), int(y_min_orig), int(x_max_orig - x_min_orig), int(y_max_orig - y_min_orig))
        )

def clean_ocr_text(text):
    text = re.sub(r'(?<=[0-9])[Oo](?=[0-9])', '0', text) # 숫자 사이의 O,o는 0으로
    text = re.sub(r'(?<=[0-9])[Il](?=[0-9])', '1', text) # 숫자 사이의 I,l은 1로
    text = re.sub(r'(\d+)[,\s.]+(\d{3})', r'\1,\2', text) # 금액 패턴 (콤마, 공백, 점)
    return text.strip()

cleaned_texts = [clean_ocr_text(text) for text in extracted_texts]

# ----------------------------------------------------------------
# 7. 라인별 텍스트 그룹화 (원본 이미지 좌표 기준)
# ----------------------------------------------------------------
# Y 좌표 기준으로 정렬 (박스의 Y 시작점 기준)
# extracted_boxes_scaled_to_original의 각 요소는 (x, y, w, h) 형태
sorted_ocr_data = sorted(zip(cleaned_texts, extracted_boxes_scaled_to_original), key=lambda x: x[1][1])

grouped_lines = []
current_line_elements = []
last_y_coordinate = -1
y_coordinate_tolerance = 15 # Y 좌표 허용 오차 (원본 이미지 기준, 실험적으로 조정 필요)

for text, (x, y, w, h) in sorted_ocr_data:
    if not text: continue # 빈 텍스트는 제외

    if last_y_coordinate == -1 or abs(y - last_y_coordinate) <= y_coordinate_tolerance:
        current_line_elements.append({'text': text, 'x': x, 'y': y, 'w': w, 'h': h})
    else:
        if current_line_elements:
            current_line_elements.sort(key=lambda item: item['x']) # 같은 라인 내에서 X 좌표로 정렬
            line_text = " ".join([elem['text'] for elem in current_line_elements])
            # 각 라인에 대한 상세 정보도 저장 (박스 좌표 등)
            grouped_lines.append({'line_text': line_text, 'elements': current_line_elements})
        current_line_elements = [{'text': text, 'x': x, 'y': y, 'w': w, 'h': h}]
    last_y_coordinate = y

if current_line_elements: # 마지막 라인 처리
    current_line_elements.sort(key=lambda item: item['x'])
    line_text = " ".join([elem['text'] for elem in current_line_elements])
    grouped_lines.append({'line_text': line_text, 'elements': current_line_elements})

print("\n=== 그룹화된 라인별 텍스트 (전처리된 이미지 OCR 기반) ===")
for i, line_info in enumerate(grouped_lines):
    print(f"라인 {i+1}: {line_info['line_text']}")

# ----------------------------------------------------------------
# 8. OCR 결과 시각화 (원본 이미지에 표시)
# ----------------------------------------------------------------
plt.figure(figsize=(12, 20))
plt.imshow(receipt_image_pil) # 원본 이미지에 결과 표시

for (x, y, w, h), text in zip(extracted_boxes_scaled_to_original, cleaned_texts):
    if not text: continue
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor="r", facecolor="none")
    plt.gca().add_patch(rect)
    display_text = text if len(text) < 15 else text[:12] + "..."
    plt.text(x, y - 5, display_text, color="blue", fontsize=7,
             bbox=dict(facecolor="white", alpha=0.6, pad=0), fontweight='bold')

plt.title("OCR 결과 (전처리된 이미지 기반, 원본에 표시)")
plt.axis('off')
plt.tight_layout()
plt.show()

# ----------------------------------------------------------------
# 9. 영수증 데이터 추출 함수 정의 및 실행
# ----------------------------------------------------------------
def extract_receipt_data_from_lines(lines_info_list):
    receipt_data = {
        "store_info_name": "", "storeinfo_bizNum": "", "storeinfo_address": [],
        "paymentinfo_date": "", "paymentinfo_time": "", "sub_Results": [],
        "totalPrice": "", "subtotal": "", "subtotal_taxprice": "", "paymentTotal": "",
        "paymentinfo_cardinfo_number": "", "paymentinfo_confirmNum": "",
        "paymentinfo_cardinfo_company": ""
    }
    all_text_lines = [info['line_text'] for info in lines_info_list]

    # 가게 이름 (일반적으로 첫 번째 또는 두 번째 라인에서 가장 긴 텍스트)
    if len(all_text_lines) > 0:
        candidate_names = all_text_lines[:2] # 처음 두 라인
        receipt_data["store_info_name"] = max(candidate_names, key=len).strip() if candidate_names else ""


    for line_text in all_text_lines:
        # 사업자 번호
        biz_num_match = re.search(r'(\d{3}-\d{2}-\d{5})', line_text)
        if biz_num_match and not receipt_data["storeinfo_bizNum"]:
            receipt_data["storeinfo_bizNum"] = biz_num_match.group(1)

        # 주소 (간단한 키워드 기반, 개선 필요)
        if any(keyword in line_text for keyword in ["주소:", "대전", "서울", "경기", "인천", "부산", "광주", "울산", "대구", "세종"]) and len(receipt_data["storeinfo_address"]) < 2 :
             address_part = re.sub(r'주소\s*:\s*', '', line_text).strip()
             if address_part: receipt_data["storeinfo_address"].append(address_part)


        # 날짜 및 시간
        date_match = re.search(r'(\d{4}[년\./-]\s*\d{1,2}[월\./-]\s*\d{1,2}일?)', line_text)
        time_match = re.search(r'(\d{1,2}:\d{2}(?::\d{2})?)', line_text)
        if date_match and not receipt_data["paymentinfo_date"]:
            receipt_data["paymentinfo_date"] = date_match.group(1)
        if time_match and not receipt_data["paymentinfo_time"]:
            receipt_data["paymentinfo_time"] = time_match.group(1)

        # 상품 목록 (매우 단순화된 버전, 정교한 로직 필요)
        # 예시: "상품명 0,000 0 00,000" 또는 "상품명 0000" 패턴
        # 이 부분은 영수증 형식에 따라 매우 복잡해질 수 있습니다.
        # 금액처럼 보이는 숫자, 수량처럼 보이는 숫자, 그리고 텍스트를 조합해야 합니다.
        # 아래는 매우 기본적인 아이템 감지 시도입니다.
        item_match = re.match(r'^(?!\s*합계)(?!\s*총액)(?!\s*결제)(?!.*\d{3}-\d{2}-\d{5})([^\d]+?)\s*([\d,]+)\s*(\d*)\s*([\d,]*)', line_text.strip())
        if item_match:
            name, p1, q, p2 = item_match.groups()
            name = name.strip()
            # p1: 단가 또는 금액, q: 수량(생략가능), p2: 금액(생략가능)
            # 더 정교한 로직으로 단가, 수량, 금액 구분 필요
            price_str = p2 if p2 else p1 # 일단 마지막 숫자를 금액으로
            count_str = q if q else "1"
            unit_price_str = p1 if p2 else "" # p2가 금액이면 p1이 단가일 가능성

            if name and len(name) > 1 and price_str: # 최소한의 조건
                receipt_data["sub_Results"].append({
                    "subResults_items_name": name,
                    "subResults_itmes_unitPrice": unit_price_str.replace(",", ""),
                    "subResults_itmes_count": count_str,
                    "subResults_itmes_price": price_str.replace(",", "")
                })

        # 합계 금액
        if any(keyword in line_text for keyword in ["합계", "총액", "총 계", "받을금액"]) and not receipt_data["totalPrice"]:
            total_match = re.search(r'([\d,]+)원?', line_text)
            if total_match:
                receipt_data["totalPrice"] = total_match.group(1).replace(",", "")
        # 결제 금액
        if "결제금액" in line_text and not receipt_data["paymentTotal"]:
            payment_match = re.search(r'([\d,]+)원?', line_text)
            if payment_match:
                receipt_data["paymentTotal"] = payment_match.group(1).replace(",", "")
        # 카드 정보
        if "카드" in line_text or "CARD" in line_text.upper():
            card_num_match = re.search(r'(\d{4}[-\s]\d{4}[-\s]\*{4}[-\s]\d{4}|\d{6}[-\s]\*{6,7})', line_text) # 카드번호 패턴
            if card_num_match and not receipt_data["paymentinfo_cardinfo_number"]:
                receipt_data["paymentinfo_cardinfo_number"] = card_num_match.group(1)

            card_company_match = re.search(r'(신한|국민|KB|롯데|삼성|현대|우리|하나|농협|BC|비씨|카카오|케이뱅크)', line_text)
            if card_company_match and not receipt_data["paymentinfo_cardinfo_company"]:
                receipt_data["paymentinfo_cardinfo_company"] = card_company_match.group(1)
        # 승인번호
        if "승인" in line_text and not receipt_data["paymentinfo_confirmNum"]:
             confirm_match = re.search(r'승인[번호NO\s:]*(\d{6,12})', line_text)
             if confirm_match : receipt_data["paymentinfo_confirmNum"] = confirm_match.group(1)


    # totalPrice가 비어있으면 paymentTotal로 채우거나 그 반대
    if not receipt_data["totalPrice"] and receipt_data["paymentTotal"]:
        receipt_data["totalPrice"] = receipt_data["paymentTotal"]
    elif not receipt_data["paymentTotal"] and receipt_data["totalPrice"]:
        receipt_data["paymentTotal"] = receipt_data["totalPrice"]


    return receipt_data

# 데이터 추출 실행 (그룹화된 라인 정보 사용)
extracted_receipt_info = extract_receipt_data_from_lines(grouped_lines)

print("\n=== 추출된 영수증 데이터 (JSON) ===")
print(json.dumps(extracted_receipt_info, ensure_ascii=False, indent=2))

# ----------------------------------------------------------------
# 10. 결과 파일로 저장
# ----------------------------------------------------------------
output_json_path = '/content/receipt_data_extracted.json'
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(extracted_receipt_info, f, ensure_ascii=False, indent=2)

print(f"\n추출된 데이터가 '{output_json_path}' 파일로 저장되었습니다.")